In [9]:

import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pavad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv(r"C:\Users\pavad\Downloads\tweets_1.csv")


<ipython-input-10-fdd2690b0de8>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [11]:
x= df['text'][df['Unnamed: 0']==2]
print(x)

2    The media and establishment want me out of the race so badly -  I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA
Name: text, dtype: object


In [12]:
df.head(10)

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA",DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz,DonaldTrump
5,5,4h4 hours ago,786950598826532864,/realDonaldTrump/status/786950598826532864,False,WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIGN’S ANTI-CATHOLIC BIGOTRY: \nhttp://bit.ly/2dcbtvkCrooked,DonaldTrump
6,6,Oct 13,786589172911964160,/realDonaldTrump/status/786589172911964161,False,Thank you! #MAGA #AmericaFirstpic.twitter.com/fG313wjlKm,DonaldTrump
7,7,Oct 12,786340623804751872,/realDonaldTrump/status/786340623804751872,False,"I will be in Cincinnati, Ohio tomorrow night at 7:30pm- join me! #OhioVotesEarly #VoteTrumpPence16 \nTickets: https://www.donaldjtrump.com/schedule/register/cincinnati-oh2/ …pic.twitter.com/XUFuGc4Fg5",DonaldTrump
8,8,Oct 12,786201435486781440,/realDonaldTrump/status/786201435486781440,False,Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system!,DonaldTrump
9,9,Oct 11,786054986534969344,/realDonaldTrump/status/786054986534969344,False,Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out & #VoteTrumpPence16 on 11/8! #MAGApic.twitter.com/e90a2AL7vO,DonaldTrump


In [13]:

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [14]:
df['text'] = df['text'].astype(str)

In [15]:
df['text'] = df['text'].astype(str)
df['cleaned'] = df['text'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)


In [16]:
EMBEDDING_FILE = r'C:\Users\pavad\Downloads/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


In [17]:
#ssplit en plusieurs mots la description cleaned

import time
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [18]:
# Entrainement surr google model
%time
google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)



google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

Wall time: 0 ns


(0, 0)

# LIMIT MODEL TRAINING


In [19]:
google_model.wv.most_similar(positive=["steve"]) #model google des mots reconus on peut tester des trucs ex: ok, hello, etc

[('dave', 0.7480319738388062),
 ('jeff', 0.6910731196403503),
 ('jason', 0.6878510117530823),
 ('robert', 0.6830393075942993),
 ('george', 0.6792417764663696),
 ('kevin', 0.6762767434120178),
 ('todd', 0.6758199334144592),
 ('ryan', 0.6735630631446838),
 ('greg', 0.6706033945083618),
 ('michael', 0.6679340600967407)]

In [20]:
# genere average word2dec pour chaque champ
def vectors(x):
    
    
    global word_embeddings
    word_embeddings = []

    
    for line in df['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)
    
    return len(word_embeddings)


In [34]:
def essentiel(df,text):
    df = df.append({'Unnamed: 0': 17216,'date':"dateeclate",'id':1,'link':"/realDonaldTrump/status/1784884848", 'retweet':"False", 'text':text,'author':"DonalTrump"}, ignore_index=True)
    df['text'] = df['text'].astype(str)
    df['cleaned'] = df['text'].apply(_removeNonAscii)
    df['cleaned'] = df.cleaned.apply(func = make_lower_case)
    df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
    df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
    df['cleaned'] = df.cleaned.apply(func=remove_html)
    return (df)
    

In [37]:

pd.set_option('display.max_colwidth', -1)
df = pd.read_csv(r"C:\Users\pavad\Downloads\tweets_1.csv")

df = essentiel(df,"Love you")

def recommendations(text):
    
    vectors(df)
    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
    books = df[['text','id']]
    indices = pd.Series(df.index, index = df['text'])
    idx = indices[text]  
    sim_scores = list(enumerate(cosine_similarities[idx]))    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:20]
    book_indices = [i[0] for i in sim_scores]
    recommend = books.iloc[book_indices]

    
    return(recommend)

<ipython-input-37-24780a29cdf1>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [38]:
recommendations("Love you")

<ipython-input-20-49c306424514>:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  avgword2vec = google_model[word]
<ipython-input-20-49c306424514>:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  avgword2vec = avgword2vec + google_model[word]


,text,id
9983,@BrikMillerEDG You will love it!,357509419464003584
12541,@RealKyleMorris You will love it!,287230122526924800
11729,"""@Candynecklace2: @realDonaldTrump I love love love Mr. Trump he is my hero "" Thanks.",307337362634076160
8862,"""@_stacys_mom_: I just love @realDonaldTrump."" Very nice.",394356629384482816
12548,@michelle88fan I love what @GovernorCorbett is doing.,286949964746211328
5392,"""@biggangVH1: @realDonaldTrump love the show! #CelebApprentice""",551935679459250176
5363,"""@loribuckmajor: @realDonaldTrump @BrandiGlanville is as real as they come. Love her #CelebApprentice""",551953704241618944
5064,"""@MelRivers: @realDonaldTrump @IvankaTrump @DonaldJTrumpJr Thank you for all of your kindness and support. My mother loved you all."" LOVE!",557405593661890560
5999,"""@LenWDougleman: Dear Donald, I love you success aftershave. I love to walk out of the house with the confidence that I smell of Trump.""",530704729731784704
13489,.@brandonhardest Love what you do and work hard.,250961000759693312
